## Setup

In [1]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.47  Python-3.12.3 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3050, 8192MiB)
Setup complete  (32 CPUs, 31.8 GB RAM, 608.3/930.7 GB disk)


In [2]:
!pip install supervision

from IPython import display
display.clear_output()

import supervision as sv
print("supervision.__version__:", sv.__version__)

supervision.__version__: 0.19.0


In [3]:
MODEL = "yolov8x.pt"
from ultralytics import YOLO

model = YOLO(MODEL)

In [4]:
results = model.train(data='cocodataset/bicycle/data.yaml',
                        device=0)

New https://pypi.org/project/ultralytics/8.2.16 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.47  Python-3.12.3 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3050, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8x.pt, data=cocodataset/bicycle/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train16, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=

train: Scanning C:\Users\ignac\Documents\ipre_ciclistas\cocodataset\bicycle\train\labels.cache... 7540 images, 138 backgrounds, 0 corrupt: 100%|██████████| 7540/7540 [00:00<?, ?it/s]

WARNING  Box and segment counts should be equal, but got len(segments) = 4276, len(boxes) = 9355. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



val: Scanning C:\Users\ignac\Documents\ipre_ciclistas\cocodataset\bicycle\valid\labels.cache... 1073 images, 43 backgrounds, 0 corrupt: 100%|██████████| 1073/1073 [00:00<?, ?it/s]

WARNING  Box and segment counts should be equal, but got len(segments) = 756, len(boxes) = 1435. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


...

    100/100        13G     0.3259     0.1952     0.8674          7        640: 100%|██████████| 472/472 [1:49:13<00:00, 13.89s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [06:44<00:00, 11.91s/it]

                   all       1073       1435      0.853      0.803      0.844      0.657



100 epochs completed in 183.979 hours.
Optimizer stripped from runs\detect\train16\weights\last.pt, 136.7MB
Optimizer stripped from runs\detect\train16\weights\best.pt, 136.7MB

Validating runs\detect\train16\weights\best.pt...
Ultralytics YOLOv8.1.47  Python-3.12.3 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3050, 8192MiB)
Model summary (fused): 268 layers, 68125494 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:35<00:00,  1.04s/it]


                   all       1073       1435      0.884      0.786      0.854      0.658
               bicycle       1073        828      0.837      0.658       0.75      0.514
              escooter       1073        607      0.931      0.914      0.958      0.803
Speed: 0.2ms preprocess, 30.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\train16


In [3]:
MODEL = "yolov8x.pt"
from ultralytics import YOLO

model = YOLO(MODEL)

In [4]:
results = model.train(data='datasets/e-scooter-bicycle-detection.v1i.yolov8/data.yaml', device=0)

New https://pypi.org/project/ultralytics/8.2.6 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.47  Python-3.12.3 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3050, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8x.pt, data=datasets/e-scooter-bicycle-detection.v1i.yolov8/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mas

train: Scanning C:\Users\ignac\Documents\ipre_ciclistas\datasets\e-scooter-bicycle-detection.v1i.yolov8\train\labels... 2716 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2716/2716 [00:01<00:00, 1901.39it/s]

train: WARNING  C:\Users\ignac\Documents\ipre_ciclistas\datasets\e-scooter-bicycle-detection.v1i.yolov8\train\images\img-8OK4R1OBTF_png_jpg.rf.cb77fe45f094d98190775a2d0489c65a.jpg: 1 duplicate labels removed


train: New cache created: C:\Users\ignac\Documents\ipre_ciclistas\datasets\e-scooter-bicycle-detection.v1i.yolov8\train\labels.cache
WARNING  Box and segment counts should be equal, but got len(segments) = 661, len(boxes) = 3807. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


val: Scanning C:\Users\ignac\Documents\ipre_ciclistas\datasets\e-scooter-bicycle-detection.v1i.yolov8\valid\labels... 738 images, 0 backgrounds, 0 corrupt: 100%|██████████| 738/738 [00:01<00:00, 673.34it/s]

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     33/100      13.2G      1.055      0.913      1.387         43        640: 100%|██████████| 170/170 [42:43<00:00, 15.08s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [05:15<00:00, 13.13s/it]

                   all        738       1054      0.787      0.756      0.796       0.54



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     34/100      13.2G     0.9657     0.8413      1.329         33        640:  27%|██▋       | 46/170 [11:44<38:40, 18.71s/it]

In [69]:

model.fuse()
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names

# class_ids of interest - car, motorcycle, bus and truck
selected_classes = [1]

YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


In [70]:
import os
import csv
from tqdm.notebook import tqdm
import numpy as np
import math
import torch
from IPython import display
import subprocess


## Utils

In [71]:
import VARIABLES
import importlib
importlib.reload(VARIABLES)
from VARIABLES import *

In [72]:
def calculate_hypotenuse(a, b):
  return math.sqrt(a**2 + b**2)
  

In [73]:
class VideoInfoHandler():
  def __init__(self) -> None:
    self.video_info = None
    self.va_params = {}
    self.line_zone_annotators = []
    self.label_annotator = None
    self.trace_annotator = None
    self.byte_tracker = None
    self.line_zones = [] 
    pass
    
  def re_init(self, SOURCE_VIDEO_PATH):
    self.video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
    self.init_va_params()
    self.init_line_zone_annotators()
    self.label_annotator = sv.LabelAnnotator( text_thickness=self.va_params["text_thickness"], text_scale=self.va_params["text_scale"])
    self.trace_annotator = sv.TraceAnnotator(thickness=self.va_params["thickness"], trace_length=self.va_params["trace_length"])
    self.byte_tracker = sv.ByteTrack(
      track_activation_threshold=0.25, lost_track_buffer=30, minimum_matching_threshold=0.8, frame_rate=self.video_info.fps
    )
    self.init_line_zones()
    
  def init_va_params(self):
    video_default_size = calculate_hypotenuse(1920, 1080)
    video_current_size = calculate_hypotenuse(self.video_info.width, self.video_info.height)
    proportion = video_current_size / video_default_size
    self.va_params = {
        "thickness": round(THICKNESS_DEFAULT * proportion),
        "text_thickness": round(TEXT_THICKNESS_DEFAULT * proportion),
        "text_scale": TEXT_SCALE_DEFAULT * proportion,
        "trace_length": round(TRACE_LENGTH_DEFAULT * proportion),
    }
    
  def init_line_zone_annotators(self):
    self.line_zone_annotators = [sv.LineZoneAnnotator(
                thickness=self.va_params["thickness"],
                text_thickness=self.va_params["text_thickness"],
                text_scale=self.va_params["text_scale"]
                )
              for _ in range(3)]
    
  def get_line_zones(self):
    line_zones = []
    for i in [-1, 0, 1]:
        x = self.video_info.width * (1 / 2 + i * 0.15)
        line_zones.append(
          sv.LineZone(
          start=sv.Point( x, 0),
          end=sv.Point(x, self.video_info.height)
          )
        )
    return line_zones
          
  def init_line_zones(self):
    new_line_zones = self.get_line_zones()
    if self.line_zones:
      for i, ex_line in enumerate(self.line_zones):
        new_line_zones[i].in_count = ex_line.in_count
        new_line_zones[i].out_count = ex_line.out_count
    
    self.line_zones = new_line_zones
    
    
    

  

In [74]:

def process_video(
    source_path: str,
    target_path: str,
    callback,
    stride=1,
) -> None:
    """
    Process a video file by applying a callback function on each frame
        and saving the result to a target video file.

    Args:
        source_path (str): The path to the source video file.
        target_path (str): The path to the target video file.
        callback (Callable[[np.ndarray, int], np.ndarray]): A function that takes in
            a numpy ndarray representation of a video frame and an
            int index of the frame and returns a processed numpy ndarray
            representation of the frame.

    Examples:
        ```python
        import supervision as sv

        def callback(scene: np.ndarray, index: int) -> np.ndarray:
            ...

        process_video(
            source_path=<SOURCE_VIDEO_PATH>,
            target_path=<TARGET_VIDEO_PATH>,
            callback=callback
        )
        ```
    """
    source_video_info = sv.VideoInfo.from_video_path(video_path=source_path)
    with sv.VideoSink(target_path=target_path, video_info=source_video_info) as sink:
        for index, frame in tqdm(enumerate(
            sv.get_video_frames_generator(source_path=source_path, stride=stride)
        ), desc=" Video processing", position=1, leave=False, total=source_video_info.total_frames):
            result_frame = callback(frame, index)
            sink.write_frame(frame=result_frame)



In [75]:
import VARIABLES
import importlib
importlib.reload(VARIABLES)
from VARIABLES import *





videos_folder = "3-3"
videos_folder_path = os.path.join("full_recordings", videos_folder)
data = [["file_name", "in", "out"]]

prev_in, prev_out = 0, 0



with open(os.path.join("results", f"{videos_folder}.csv"), "w", newline="") as csv_output:
  writer = csv.writer(csv_output)
  try:
    for file_name in tqdm(os.listdir(videos_folder_path), desc=" Videos remaining", position=0):
      file_path = os.path.join(videos_folder_path, file_name)
      vih.re_init(file_path)
      process_video(
        source_path = file_path,
        target_path = TARGET_DUMMY_VIDEO_PATH,
        model=model,
        selected_classes=selected_classes,
        callback=callback,
      )
      max_in = max(vih.line_zones, key=lambda x: x.in_count)
      max_out = max(vih.line_zones, key=lambda x: x.out_count)
      
      data.append([file_name, max_in.in_count - prev_in, max_out.out_count - prev_out])
      prev_in = max_in.in_count
      prev_out = max_out.out_count
      
  except KeyError as e:
    print(e)
  finally: 
    writer.writerows(data)
    result = subprocess.run(f"echo {max_in.in_count}, {max_out.out_count} > results/{videos_folder}.txt", shell=True, capture_output=True, text=True)

    # Check if the command was successful
    if result.returncode == 0:
        print(result.stdout)
    else:
        print("Command failed with error:")
        print(result.stdout)
        print(result.stderr)
    print(max_in.in_count, max_out.out_count)



 Videos remaining:   0%|          | 0/5 [00:00<?, ?it/s]

 Video processing:   0%|          | 0/18032 [00:00<?, ?it/s]

 Video processing:   0%|          | 0/18031 [00:00<?, ?it/s]

 Video processing:   0%|          | 0/18032 [00:00<?, ?it/s]

 Video processing:   0%|          | 0/18031 [00:00<?, ?it/s]

 Video processing:   0%|          | 0/2682 [00:00<?, ?it/s]


153 114


In [1]:
import subprocess

a, b = 2, 5
result = subprocess.run(f"echo {a} {b} > results/3-2.txt", shell=True, capture_output=True, text=True)

# Check if the command was successful
if result.returncode == 0:
    print(result.stdout)
else:
    print("Command failed with error:")
    print(result.stdout)
    print(result.stderr)